# FinanceDataReader Tutorial
## 섹터 평균 수익률과 개별 종목의 수익률 구하기

<img width="320px" src="https://i.imgur.com/r0YE5Xs.png" ><br>
https://github.com/FinanceData/FinanceDataReader


#### 2018-2020 [FinanceData.KR](http://financedata.kr) | [facebook.com/financedata](http://facebook.com/financedata)


## 요약
* KRX 전체종목과 섹터정보를 가져옵니다
* groupby()를 활용하여 섹터별로 묶어 냅니다
* groupby()에 간단한 통계함수를 적용하여 count, sum, mean 등을 구할 수 있습니다
* 섹터 개별 종목들의 데이터를 수집하고 종가(Close)를 취합 합니다
* 섹터종목의 들의 개별 수익률과 섹터 전체의 평균 수익률을 계산합니다
* 기간을 지정하여(timedelta 활용) 다양한 기간의 수익률을 계산합니다
* 데이터프레임 스타일을 적용하여 주요 셀을 강조합니다
* 스타일을 적용한 데이터프레임을 엑셀로 저장합니다


## FinanceDataReader 설치

In [0]:
!pip -q install finance-datareader

## 한국거래소(KRX)의 전체 종목 가져오기

In [0]:
import FinanceDataReader as fdr

krx = fdr.StockListing('KRX')
krx

`groupby('컬럼명')`을 사용하여 컬럼명을 기준으로 묶어 줄 수 있습니다.

In [0]:
krx.groupby('Sector')

groupby 결과에 통계함수(count, mean, sum 등)을 적용해주면 묶은 그룹별 통계량을 산출할 수 있습니다.

In [0]:
krx.groupby('Sector').count()

섹터(Sector)의 개수는 모두 156개 인것을 확인할 수 있습니다.

섹터(Sector)별로 개수(종목수)를 구하고, 가장 종목수가 많은 섹터 30개를 뽑아 봅니다.

In [0]:
krx.groupby('Sector').count().sort_values('Symbol', ascending=False)[:30]

묶은(groupby)결과를 list와 dict로 변환하면 개별 섹터(Sector)별로 종목리스트를 구분할 수 있습니다.

In [0]:
sectors = dict(list(krx.groupby('Sector')))

print('count:', len(sectors))
list(sectors.keys())[:10]

In [0]:
med_sec = sectors['의료용품 및 기타 의약 관련제품 제조업']

print('row count:', len(med_sec))
med_sec.head(10)

## FinanceDataReader 설치

In [0]:
!pip -q install finance-datareader

## 개별 종목의 가격 데이터 가져오기
개별 종목의 가격을 가져오고, 개별 종목의 종가(Close)를 데이터 프레임(med_pirces)에 컬럼으로 추가합니다.

In [0]:
import pandas as pd
import FinanceDataReader as fdr

med = pd.DataFrame()

for ix, row in med_sec.iterrows():
  code, name = row['Symbol'], row['Name']
  print(code, name)
  # 개별 종목의 가격을 가져옵니다
  df = fdr.DataReader(code, '2019-01-01', '2019-12-31')

  # 가격 데이터의 종가(Close)를 컬럼으로 추가합니다
  # (컬럼명은 종목명을 지정합니다)
  med[name] = df['Close']

In [0]:
# 23개 종목 1년간의 종가(Close) 데이터를 수집하였습니다
med

2019년 중에 신규상장한 2개 종목은 제외합니다. (1년간 기간 수익률을 구하는 것이므로)
* 제테마(코스닥, 2019년 11월 14일 신규상장)
* 원바이오젠(코넥스, 2019년 6월 3일 신규상장)

결측치(NaN)이 포함된 컬럼을 제외합니다.

In [0]:
med = med.dropna(axis=1)
med

## 수익률 계산
데이터의 첫날(1월 2일)을 기준으로 일일 누적 수익률을 구하기 위해 다음과 같이 연산합니다.


In [0]:
acc_rets = med / med.iloc[0] - 1.0
acc_rets

종목별 12월 30일의 최종 수익률은 마지막 행(row)가 됩니다.

In [0]:
returns = acc_rets.iloc[-1]
returns

수익률이 큰 종목부터 나열해 봅니다

In [0]:
returns.sort_values(ascending=False)

2019년 1년의 연간 수익률에서 가장 높은 수익률을 보인 것은 씨젠 96.4% 이고, 가장 낮은 수익률을 보인것은 수젠텍 -57.5% 입니다.

In [0]:
returns.mean()

21개 종목의 평균수익률은 -7.4% 입니다.

## 다양한 기간에 대한 수익률

기간을 달리해서 수익률을 추출해 봅니다.

In [0]:
df = med['2019-12-01':'2019-12-30'] # 특정 기간(12월 1달) 동안
acc_rets = df / df.iloc[0] - 1.0
acc_rets.iloc[-1]

'2019-12-30' 시점을 기준으로 과거 5일, 10일, 20일, 60일, 120일, 240일 각각 수익률을 구해봅니다.

각 기간의 (시작날짜, 끝날짜)는 다음과 같이 생성할 수 있습니다.

In [0]:
from datetime import datetime, timedelta

the_day = datetime(2019, 12, 30)

for days in [5, 10, 20, 60, 120, 240]:
  start = the_day - timedelta(days)
  end = the_day
  print(start, '~', end)

In [0]:
from datetime import datetime, timedelta

the_day = datetime(2019, 12, 30)
row_dict = {}
for days in [5, 10, 20, 60, 120, 240]:
  start = the_day - timedelta(days)
  end = the_day

  df = med[start:end] # 특정 기간
  acc_rets = df / df.iloc[0] - 1.0
  row_dict[days] = acc_rets.iloc[-1] 

In [0]:
df_rets = pd.DataFrame(row_dict)
df_rets

## 데이터프레임 스타일

In [0]:
def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

df_rets.style.\
    applymap(color_negative_red).\
    apply(highlight_max)

In [0]:
## 엑셀로 저장
df_rets.style.\
    applymap(color_negative_red).\
    apply(highlight_max).\
    to_excel('기간별수익률데이터(2019.12.30).xlsx', engine='openpyxl')

In [0]:
# 구글 colab 에서 실행한 경우 엑셀 파일을 다음과 같이 다운로드 합니다.

from google.colab import files
files.download('기간별수익률데이터(2019.12.30).xlsx')

#### 2020 [FinanceData.KR](http://financedata.kr) | [facebook.com/financedata](http://facebook.com/financedata)